In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# Import Dependencies

In [ ]:
import os
import random
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from matplotlib.image import imread
%config InlineBackend.figure_format = 'svg'

# Selecting the folder

In [ ]:
data_dir = '/classification_dataset/'

In [ ]:
train_path = data_dir + 'train/'
test_path = data_dir + 'test/'

In [ ]:
print(os.listdir(train_path))
print(os.listdir(test_path))

In [ ]:
imageName = os.listdir(train_path + '0')[0]

# Viewing an example image of the knee in the dataset

In [ ]:
img = (train_path + '0/' + imageName)
plt.imshow(imread(img), cmap = 'gray')

# Counting the amount of data in each severity level

In [ ]:
# train
for i in os.listdir(train_path):
    print(f'{i}: {len(os.listdir(train_path + i))}')

In [ ]:
# test
for i in os.listdir(test_path):
    print(f'{i}: {len(os.listdir(test_path + i))}')

# Data Augmentation

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
image_gen = ImageDataGenerator(brightness_range = [0.2, 1.0],
                                rotation_range = 20,
                                rescale = 1 / 255,
                                shear_range = 0.1,
                                zoom_range = 0.1,
                                horizontal_flip = True,
                                vertical_flip = True,
                                fill_mode = 'nearest')

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.layers import Conv2D, MaxPool2D, MaxPooling2D, BatchNormalization

In [ ]:
BATCH_SIZE = 16
IMG_SHAPE = (160, 335, 1)

In [ ]:
train_image_gen = image_gen.flow_from_directory(train_path,
                                                target_size = IMG_SHAPE[:2],
                                                color_mode = 'rgb',
                                                batch_size = BATCH_SIZE,
                                                class_mode = 'categorical',
                                                shuffle = True)

In [ ]:
test_image_gen = image_gen.flow_from_directory(test_path,
                                                target_size = IMG_SHAPE[:2],
                                                color_mode = 'rgb',
                                                batch_size = BATCH_SIZE,
                                                class_mode = 'categorical',
                                                shuffle = False)

In [ ]:
train_image_gen.class_indices

# Creating Callbacks

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger

In [ ]:
# Early Stop
early_stop = EarlyStopping(monitor='val_accuracy',patience=10, verbose=2, restore_best_weights=True)

In [ ]:
# Checkpoint Saving
saveCheckpointsPath = "/models/XceptionModels/improved-{epoch:02d}-{val_accuracy:.2f}.keras"
Xceptioncheckpoint = ModelCheckpoint(saveCheckpointsPath, monitor = 'val_accuracy', verbose = 1, save_best_only = True, mode = 'max')

In [ ]:
# Record Accuracy and Loss in a CSV file
logXception = CSVLogger("XceptionLogs.csv", separator = ',', append = False)

# Calculating the class weights

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
y_train = train_image_gen.labels
y_test = test_image_gen.labels

In [ ]:
class_weights = compute_class_weight(
    class_weight="balanced", classes=np.unique(y_train), y=y_train
)
train_class_weights = dict(enumerate(class_weights))
print(train_class_weights)

# Xception Model

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [ ]:
# Define the input shape
input_shape = (160, 335, 3)
inputs = tf.keras.layers.Input(shape=input_shape)

# Load the Xception model with the defined input tensor
XceptionModel = tf.keras.applications.Xception(
    input_tensor=inputs,
    include_top=False,
    weights='imagenet'
)

# Set all layers to be trainable
for layer in XceptionModel.layers:
    layer.trainable = True


X = XceptionModel.output
# custom layers
X = tf.keras.layers.GlobalAveragePooling2D()(X)
X = Dropout(0.2)(X)
predictions = Dense(5, activation='softmax')(X)

Xception_model = Model(inputs=XceptionModel.input, outputs=predictions)

# summary
Xception_model.summary()

# compile
Xception_model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model Training

In [ ]:
history = Xception_model.fit(
    train_image_gen,
    epochs=100,
    callbacks=[early_stop, Xceptioncheckpoint, logXception],
    validation_data=test_image_gen,
    class_weight=train_class_weights,
    verbose=1,
)

# Saving the model

In [ ]:
Xception_model.save('models/XceptionModel.keras')

In [ ]:
Xception_model.metrics_names

# Evaluate model with test data

In [ ]:
Xception_model.evaluate(test_image_gen)

# Visualize the plot of the result

In [ ]:
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

train_loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(10, 6))

ax[0].set_title('Training/Validation Accuracy vs Epochs')
ax[0].plot(train_accuracy, 'o-', label='Training Accuracy')
ax[0].plot(val_accuracy, 'o-', label='Validation Accuracy')
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('Accuracy')
ax[0].legend(loc='best')

ax[1].set_title('Training/Validation Loss vs Epochs')
ax[1].plot(train_loss, 'o-', label='Training Loss')
ax[1].plot(val_loss, 'o-', label='Validation Loss')
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Loss')
ax[1].legend(loc='best')

plt.tight_layout()
plt.show()

# Loading the last Checkpoint with the best accuracy

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
LoadedModel = load_model('/models/XceptionModels/improved-48-0.84.keras')

# Evaluating the Model Loaded with the Test Data

In [ ]:
LoadedModel.metrics_names

In [ ]:
LoadedModel.evaluate(test_image_gen)

# Plotting the graphs

In [ ]:
df = pd.read_csv('XceptionLogs.csv')

In [ ]:
train_accuracy = df['accuracy']
val_accuracy = df['val_accuracy']

train_loss = df['loss']
val_loss = df['val_loss']

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(10, 6))

ax[0].set_title('Training/Validation Accuracy vs Epochs')
ax[0].plot(train_accuracy, 'o-', label='Train Accuracy')
ax[0].plot(val_accuracy, 'o-', label='Validation Accuracy')
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('Accuracy')
ax[0].legend(loc='best')

ax[1].set_title('Training/Validation Loss vs Epochs')
ax[1].plot(train_loss, 'o-', label='Train Loss')
ax[1].plot(val_loss, 'o-', label='Validation Loss')
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Loss')
ax[1].legend(loc='best')

plt.tight_layout()
plt.savefig('SavedXception.png')
plt.show()

# Confusion matrix and Classificat

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
pred = LoadedModel.predict(test_image_gen)

In [ ]:
y_pred = np.argmax(pred, axis = 1)
y_test = test_image_gen.classes

In [ ]:
# print classification report
print(classification_report(y_test, y_pred, target_names=labels.values()))

In [ ]:
# plot confusion matrix
sns.heatmap(confusion_matrix(y_test, y_pred), annot = True, linecolor = 'white', cmap = 'Blues')

# Making inference on unseen values

In [ ]:
import cv2

In [ ]:
dictionary = {
    0 : "Normal",
    1 : "Doubtful",
    2 : "Mild",
    3 : "Moderate",
    4 : "Severe",
}

In [ ]:
def CheckSeverity(img):
    resize = tf.image.resize(img, (160, 335))
    plt.imshow(resize.numpy().astype(int))
    plt.show()

    prediction = LoadedModel.predict(np.expand_dims(resize / 255, 0))
    prediction = np.argmax(prediction, axis = 1)
    if prediction[0] in dictionary:
        print(dictionary[prediction[0]])

In [ ]:
img = cv2.imread('/classification_dataset/test/4/9012867R.png')
CheckSeverity(img)